In [1]:
import csv
from tqdm import tqdm

In [2]:
with open("../data/sentences.csv", "r", encoding="utf-8") as f:
    data = list(csv.DictReader(f=f, delimiter=","))

In [3]:
from flair.data import Sentence
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english")
sentence = Sentence(data[0]["sentence"])

# predict NER tags
tagger.predict(sentence)

# print sentence
print(sentence)

# print predicted NER spans
print('The following NER tags are found:')

# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(f'entity.text is: "{entity.text}"')
    print(f'entity.start_position is: "{entity.start_position}"')
    print(f'entity.end_position is: "{entity.end_position}"')
    
    # also print the value and score of its "ner"-label
    print(f'entity "ner"-label value is: "{entity.get_label("ner").value}"')
    print(f'entity "ner"-label score is: "{entity.get_label("ner").score}"\n')

C:\Users\CSA\Anaconda3\envs\ner_vasari\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-09-06 14:26:48,169 loading file C:\Users\CSA\.flair\models\ner-english\4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2023-09-06 14:26:49,640 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
Sentence: "Finally , Pope Clement , having determined that Buonarroti should return to Florence to finish the works of the sacristy and library of S . Lorenzo , gave him orders , since many statues were wanting there , as will be told in the Life of Michelagnolo himself , that he should avail himself of the most able men that could be found , and particularly of Fra Giovanni Agnolo , employing the same methods as had been adopted by Antonio da San Gallo in order to finish the works of the Madonna di Loreto . Having therefore made his way with the Frate to Florence , Miche

In [4]:
from nltk.tokenize import sent_tokenize

output = []

pbar = tqdm(total=len(data))
for sample in data:
    doc_id = sample["id"]
    text = sample["sentence"]
    sentences = [s for s in sent_tokenize(text)]
    curr_pos = 0
    for sentence_id, sentence in enumerate(sentences):
        if sentence_id>0:
            curr_pos = curr_pos + len(sentences[sentence_id-1])+1
        sentence = Sentence(sentence)
        tagger.predict(sentence)
        for entity in sentence.get_spans("ner"):
            output.append({
                "doc_id":doc_id,
                "doc_start_pos":curr_pos+entity.start_position,
                "doc_end_pos":curr_pos+entity.end_position,
                "sentence_id":sentence_id,
                "sent_start_pos":entity.start_position,
                "sent_end_pos":entity.end_position,
                "surface":entity.text,
                "type":entity.get_label("ner").value,
                "score":entity.get_label("ner").score
            })
    pbar.update(1)
pbar.close()

keys = output[0].keys()
a_file = open("../results/flair_ner/output.csv", "w", encoding="utf-8")
dict_writer = csv.DictWriter(a_file, keys)
dict_writer.writeheader()
dict_writer.writerows(output)
a_file.close()
    

100%|██████████████████████████████████████████████████████████████████████████████████| 55/55 [09:24<00:00, 10.26s/it]
